# Assignment 6

In [1]:
import numpy as np
import sympy as sym
import matplotlib as mp
import matplotlib.pyplot as plt
import random

from sympy.physics.mechanics import *
from IPython.display import Math, display
from scipy.constants import k
from scipy.ndimage import convolve, generate_binary_structure

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Weather in the land of Oz 

> Oz nevers have two nice days in a row

> If it is a nice day, there will be a 50% chance of rain or 50% chance of snow the next day

> If it is a snow or rain day, there is a 50% chance of having the same, and 25% for the other weather

We can model this using a stochastic matrix (P):

$$P = \frac{1}{4} \begin{bmatrix}
    0 & 2 & 2 \\
    1 & 2 & 1 \\
    1 & 1 & 2 
\end{bmatrix}^{}$$


In [2]:
C = sym.Rational(1, 4) # Constant factored out from our stochastic matrix
P = sym.Matrix([[0, 2, 2], [1, 2, 1], [1, 1, 2]]) # Matrix of possibilities (our stochastic matrix)
x1 = sym.Matrix([1, 0, 0]).T # Vector for good day
x2 = sym.Matrix([0, 1, 0]).T # Vector for rainy day
x3 = sym.Matrix([0, 0, 1]).T # Vector for snow day

def round_expr(expr, num_digits):
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(sym.Number)})

# Returns the powers of the stochastic matrix
def Prob(n, A=C, B=P):
    a = C ** n
    b = B ** n 
    c = a*b
    d1 = Math(r'P^{0} ='.format({n}) + sym.latex(a) + sym.latex(b))
    d2 = Math(r'P^{0} ='.format({n}) + sym.latex(c))
    d3 = Math(r'P^{0} ='.format({n}) + sym.latex(round_expr(c, 4)))
    return d1, d2, d3

# Returns the possibilities of weather on a given day 
def Loc_prob(n, X, A=C, B=P):
    a = C ** n
    b = X * B ** n 
    c = a*b
    d1 = Math(r'x_{0} ='.format({n}) + sym.latex(a) + sym.latex(b))
    d2 = Math(r'x_{0} ='.format({n}) + sym.latex(c))
    d3 = Math(r'x_{0} ='.format({n}) + sym.latex(round_expr(c, 4)))
    return d1, d2, d3

### a) Weather two days after a good day

To find the possibility of a nice day two days after a nice day, we will multiply a vector representing a nice day ($\vec{x}$) by $P^2$: 

$$\vec{x}(t=2) = \vec{x}(t=0) P^2$$

$$\vec{x}(t=0) = [1 \ \  0 \ \ 0]$$

In [3]:
Loc_prob(2, x1, C, P)[1]

<IPython.core.display.Math object>

We can see that if the first day was Monday, and it happened to be a nice day, the chances of a nice day on Wendsday is $25\%$.

### b) Fraction of days that rain in Oz

To evaluate the probability distribution of the weather, hence the fraction of the days of a certain weather occuring, we take the limit of $P^n$ as $n \rightarrow \infty$:

$$\lim_{x \rightarrow \infty} P^n = A D^n A^{-1}$$

Where: 

> $A$ is the matrix consisting of the eigenvectors of $P$

> $D$ is a diagonlized matrix of $A$ ($D = A^{-1} P A$)

In [4]:
P0 = C*P
A, D = P0.diagonalize() # Find eigenvector matrix and diagonalize P
display(Math(r'A =' + sym.latex(A)))
display(Math(r'D =' + sym.latex(D)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

We can see that:
$$\lim_{n \rightarrow \infty} D^n =
\begin{bmatrix}
    0 & 0 & 0 \\
    0 & 0 & 0 \\
    0 & 0 & 1
\end{bmatrix}$$

$$\therefore \lim_{n \rightarrow \infty} P^n = A^{-1}\begin{bmatrix}
    0 & 0 & 0 \\
    0 & 0 & 0 \\
    0 & 0 & 1
\end{bmatrix} A$$

In [5]:
Dn = sym.Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 1]])
Pn = A * Dn * A**(-1)
display(Math(r'\lim_{n \rightarrow \infty} P^n =' + sym.latex(Pn)))

<IPython.core.display.Math object>

As we can see, Oz rains $\frac{2}{5}$ of the days.

\newpage
## Ising model in one dimension 

$$E = - J \sum_{n=1}^N \sigma_n \sigma_{n+1}$$

Where $\sigma$ can be either $+1$ or $-1$.

### a) The partition function $Z$

$$Z = \sum_\mu e^{-\beta E_\mu} = \sum_{\sigma_i} e^{\beta J \sum_{n=1}^N \sigma_n \sigma_{n+1}}$$

Let's define a transfer matrix $T$: 

$$T = \begin{bmatrix}
    e^{\beta J} & e^{-\beta J} \\
    e^{-\beta J} & e^{\beta J}
\end{bmatrix}$$

> Where the indicies correspond to $\sigma_n$ and $\sigma_{n+1}$

The partition fuction then becomes: 

$$\begin{align}
Z &= \sum_{\sigma_i} \prod_{n=1}^N \vec{\sigma}_n T \vec{\sigma}_{n+1} \\
    &= \sum_{\sigma_i} \vec{\sigma}_1 T \vec{\sigma}_{n+1} \\
    &= \sum_{\sigma_1} \vec{\sigma}_1 T \vec{\sigma}_{1} \\ 
    &= \text{Tr}(T^N) \\
    &= \text{Tr}(A D^N A)\\
    &= \text{Tr}(D^N) \\
    &= \sum_i \lambda_i^N
    \end{align}$$

Where: 

> $D$ is the diagonalized matrix of $T$

> $A$ is the matrix containing the eigenvertors of $T$

> $\lambda_i$ are the eigen values of T

In [6]:
# Define the matrix T
bet, J, N = sym.symbols('beta J N')
T = sym.Matrix([[sym.exp(bet * J), sym.exp(-bet*J)], [sym.exp(-bet*J), sym.exp(bet * J)]])

In [7]:
# Find the eigenvalues and eigenvectors 
T_evec, T_eval = T.diagonalize()
display(Math(r'D =' + sym.latex(T_eval)))

<IPython.core.display.Math object>

Now that we have the eigenvalues, we can see that the partition function: 

$$\begin{align}
Z &= \left(-\sqrt{e^{-2J\beta}} + e^{J\beta} \right)^N + \left(\sqrt{e^{-2J\beta}} + e^{J\beta} \right)^N \\
  &= \left(-e^{-J\beta} + e^{J\beta} \right)^N + \left( e^{-J\beta} + e^{J\beta} \right)^N \\
  &= 2 \cosh^N(J\beta) + 2 \sinh^N(J\beta)
  \end{align}$$

### b) Graph of the heat capacity ($C_V$) as a function of temperature ($T$) for large $N$

$$C_V = \left(\frac{\partial U}{\partial T} \right)_V = -k \beta^2 \left(\frac{\partial^2 \ln(Z)}{\partial \beta^2}\right)_V$$


$$\ln(Z) = \ln \left[ 2 \cosh^N(J\beta) + 2 \sinh^N(J\beta) \right]$$

In [8]:
Z = 2*sym.cosh(J*bet)**N + 2*sym.sinh(J*bet)**N
ln_Z = sym.ln(Z)

In [9]:
dd_ln_Z = sym.diff(ln_Z, bet, 2).simplify()
display(Math(r'\frac{\partial^2 \ln(Z)}{\partial \beta^2} =' + sym.latex(dd_ln_Z)))

<IPython.core.display.Math object>

$$\frac{\partial^2 \ln(Z)}{\partial \beta^2} = J^2 N \frac{(N-1)\sinh^N(J\beta)\cosh^{N-2}(JB) + \cosh^{2(N-1)}(J\beta) - 1}{\sinh^2(J\beta)\left[\sinh^N(J\beta) + \cosh^N(J\beta)\right]^2}$$

$$\therefore C_V = -k \beta^2 J^2 N \frac{(N-1)\sinh^N(J\beta)\cosh^{N-2}(JB) + \cosh^{2(N-1)}(J\beta) - 1}{\sinh^2(J\beta)\left[\sinh^N(J\beta) + \cosh^N(J\beta)\right]^2}$$

In [10]:
def beta(T, K):
    b = 1/(K*T)
    return b

def Cv(T, K=k, J=1, N=100):
    SH = np.sinh(J*beta(T, K))
    CH = np.cosh(J*beta(T, K))
    coeff = -K * beta(T, K)**2 * J**2 * N            
    numer = (N-1) * SH**N * CH**(N-2) + CH**(2*N-2) - 1
    denum = SH**2 * (SH**N + CH**N)**2
    return -coeff * numer/denum

In [11]:
T = np.arange(0.5, 50, 0.001)

plt.figure(figsize=(10, 4))
plt.plot(T, Cv(T, K=1, N=100))
plt.title(r'Heat capacity ($C_V$) as a function of temperature ($T$)', fontsize=14)
plt.xlabel('Temperature (T) [K]', fontsize=12)
plt.ylabel(r'$\frac{C_V}{k\beta^2 J^2}$', rotation=0, fontsize=12)
plt.show()

<Figure size 720x288 with 1 Axes>

\newpage
## Ising model in two dimensions

In [12]:
L0 = np.random.randint(0, 2, size=(50, 50))*2 - 1

In [13]:
def E(A):
    A1 = np.roll(A, 1, axis=0)
    A2 = np.roll(A, -1, axis=0)
    A3 = np.roll(A, 1, axis=1)
    A4 = np.roll(A, -1, axis=1)
    B = A * (A1 + A2 + A3 + A4)
    E = B.sum(axis=0).sum()
    return -E

def flip(A):
    x_indices = np.random.randint(0, np.shape(A)[0])
    y_indices = np.random.randint(0, np.shape(A)[1])
    B = np.zeros((np.shape(A)[0], np.shape(A)[1])) + A
    B[x_indices, y_indices] = -B[x_indices, y_indices]
    return B

def accept(dE, T, K=k):
    return random.random() < np.exp(-(1/(K*T) * dE))

def mag(A):
    m = A.sum(axis=0).sum()
    N = np.shape(A)[0]*np.shape(A)[1]
    return m/N

In [14]:
def Monte(A, n, T):
    Energy = np.zeros(n)
    Magnet = np.zeros(n)
    Energy[0] = E(A)
    Magnet[0] = mag(A)
    for i in range(1, n):
        L1 = flip(A)
        if E(L1) <= E(A):
            A = L1
        else:
            dE = E(L1)-E(A)
            Accept = accept(dE, T)
            if Accept == True:
                A = L1
            else:
                A = A
        Energy[i] = E(A)
        Magnet[i] = mag(A)
    return Magnet, Energy

In [16]:
M1, E1 = Monte(L0, 20000, 1)
M2, E2 = Monte(L0, 20000, 100)
M3, E3 = Monte(L0, 20000, 200)
M4, E4 = Monte(L0, 20000, 500)
M5, E5 = Monte(L0, 20000, 1000)

In [17]:
plt.plot(M1)
plt.plot(M2)
plt.plot(M3)
plt.plot(M4)
plt.plot(M5)
plt.xlabel(r'=\beta J')
plt.ylabel(r'Mean magenetization ($\bar{m}$)')
plt.show()

<Figure size 432x288 with 1 Axes>

In [18]:
plt.plot(E1)
plt.plot(E2)
plt.plot(E3)
plt.plot(E4)
plt.plot(E5)

<Figure size 432x288 with 1 Axes>

In [19]:
np.exp(E(L0)/(100*k))

inf

In [20]:
type(k)

float